In [16]:
import warnings
import igraph as ig
import networkx as nx
import collections
import scipy as sp
import numpy as np
import bMatching as bm
import  Evaluation as ev
import importlib
import matplotlib.pylab as plt
import time
import pandas as pd
import os
import subprocess

In [ ]:
def read_graphml(f1,f2,fl=None,input_dir=None):
    
    if input_dir is not None:
        f1=input_dir+f1
        f2=input_dir+f2
        
    
    G1 = ig.read(f1,format="graphml")
    G2 = ig.read(f2,format="graphml")
    
    L=None
    if fl is not None:
        if input_dir is not None:
            fl=input_dir+fl
        L=ig.read(fl,format="graphml")
    
    return G1,G2,L

In [ ]:
def read_networks(f1,f2,fl=None,input_dir=None):
    
    if input_dir is not None:
        f1=input_dir+f1
        f2=input_dir+f2
        
    T=nx.read_leda(f1)
    nx.write_graphml(T,'graph.graphml')
    G1 = ig.read('graph.graphml',format="graphml")
    
    T=nx.read_leda(f2)
    nx.write_graphml(T,'graph.graphml')
    G2 = ig.read('graph.graphml',format="graphml")
    
    L=None
    if fl is not None:
        if input_dir is not None:
            fl=input_dir+fl
        L=ig.read(fl,format="graphml")
    
    return G1,G2,L

In [ ]:
def get_alignment(G1,G2,M):
    
    n1=G1.vcount()
    n2=G2.vcount()
    Align=set([])
   
    for (i,j) in M:    
        #print(i,j)
        u=G1.vs[i]['id']
        v=G2.vs[j]['id'] 
        #print(u,v)
        Align.add((u,v))
   
    return Align 

In [ ]:
def save_alignment(L,filename):
    
    fl=open(filename,"w")
    for (u,v) in L:
        t=u+" "+v+"\n"
        fl.write(t)
    
    fl.close()

In [ ]:
def evaluate(AQ,G1,G2,AL):
    
    AQ.mapping_set=AL
    if AQ.true_mapping_set == None:
        qual=AQ.evaluate(False,False,False,False,False,True,False,False,False,False)
        score=qual["NCV-GS3"]
    else:
        qual=AQ.evaluate(False,False,True,False,False,True,False,False,False,False)
        score=float(qual["F-NC"])
    
    return score, qual

In [ ]:
def igraph2mtx(G,n,fname='graph.mtx',bipartite=False):
    c1='%%MatrixMarket matrix coordinate real general'
    c2='% Generated'
    
    edgeG=G.get_edgelist()
    #print(edgeG)
    m=int(len(edgeG))
    
    try:
        weights=G.es['weight']
    except:
        weights=[1]*m

    
    if bipartite:
        nl=n
        nr=G.vcount()-nl
        h=str(nl)+" "+str(nr)+" "+str(m)
        
    else:
        nl=n
        nr=n
        h=str(nl)+" "+str(nr)+" "+str(m*2)
    
    f=open(fname,'w')
    
    f.write(c1+"\n")
    f.write(c2+"\n")
    f.write(h+"\n")
    #print(c1)
    #print(c2)
    #print(h)
    for i in range(m):
        (u,v)=edgeG[i]
        w=weights[i]
        
        if bipartite:
            v=v-nl
            #print(u,v,w)
            f.write(str(u+1)+" "+str(v+1)+" "+str(w)+"\n")
        else:
            #print(u,v,w)
            #print(v,u,w)
            f.write(str(u+1)+" "+str(v+1)+" "+str(w)+"\n")
            f.write(str(v+1)+" "+str(u+1)+" "+str(w)+"\n")
    
    f.close()

In [ ]:
def netAlignSC(f1,f2,fl,alpha=1,beta=1,ft=None,input_dir=None):

    G1,G2,L=read_networks(f1,f2,fl,input_dir)

    igraph2mtx(G1,G1.vcount(),fname='graph-A.mtx')
    igraph2mtx(G2,G2.vcount(),fname='graph-B.mtx')
    igraph2mtx(L,G1.vcount(),fname='graph-L.mtx',bipartite=True)


    #### Initialize evaluation class
    if ft is not None:
        AQ=ev.AlignmentQuality(input_dir+f1, input_dir+f2, None, input_dir+ft, None, None)
    else:
        AQ=ev.AlignmentQuality(input_dir+f1, input_dir+f2, None, None, None, None)

    #!../netalignmc/netalign/netAlign graph -n 40 -a $alpha -b $beta -o sc.aln
    os.system('/home/khan242/netAlign/netalignmc/netalign/netAlign graph -n 40 -v')

    f=open('sc.aln','r')
    M=[]
    for l in f:
        l=l.strip('\n')
        l=l.split(' ')
        M.append((int(l[0])-1,int(l[1])-1))
    f.close()
    AL=get_alignment(G1,G2,M)
    s,q=evaluate(AQ,G1,G2,AL)
    #!rm sc.aln graph-*  
    
    return AL,s,q

In [30]:
def netAlignSG(f1,f2,fl,alpha=1,beta=1,ft=None,input_dir=None):

    
    #!../netalignmc/netalign/netAlign graph -n 40 -a $alpha -b $beta -o sc.aln
    command=['/home/khan242/netAlign/netalignmc/netalign/netAlign', '/home/khan242/netAlign/netalignmc/data-mtx/lcsh-small', '-v', '-osc.aln']
    result = subprocess.run(command, stdout=subprocess.PIPE)
    print(result.stdout.decode('utf-8'))
    f=open('sc.aln','r')
    M=[]
    for l in f:
        l=l.strip('\n')
        l=l.split(' ')
        M.append((int(l[0])-1,int(l[1])-1))
    f.close()
    
    return M

In [ ]:
input_dir="/home/khan242/netAlign/data/synthetic_networks/"

f1="yeast0_Y2H1.gw"
f2="yeast5_Y2H1.gw"
fl="L_0.25_yeast0_yeast5.graphml"
ft="true_node_mapping.txt"

G1,G2,L=read_networks(f1,f2,fl,input_dir)

igraph2mtx(G1,G1.vcount(),fname='graph-A.mtx')
igraph2mtx(G2,G2.vcount(),fname='graph-B.mtx')
igraph2mtx(L,G1.vcount(),fname='graph-L.mtx',bipartite=True)
# AL,s,q=netAlignSC(f1,f2,fl,alpha,beta,ft,input_dir=input_dir)

# for alpha in [0,0.25,0.50,0.75,1.00]:
    
#     beta=1-alpha
#     AL,s,q=netAlignSC(f1,f2,fl,alpha,beta,ft,input_dir=input_dir)
#     save_alignment(AL,res_dir+resf)
#     print(s,q)    



In [31]:
input_dir="/home/khan242/netAlign/data/synthetic_networks/"

f1="yeast0_Y2H1.gw"
f2="yeast5_Y2H1.gw"
fl="L_0.25_yeast0_yeast5.graphml"
ft="true_node_mapping.txt"


M=netAlignSG(f1,f2,fl)
print(M)
  

finalize: 1
alpha: 1
beta: 1
gamma: 0.99
maxiter: 40
algorithm: mp
damping: 2
batch: 4
approx: 1
limitthreads: 0
problem: /home/khan242/netAlign/netalignmc/data-mtx/lcsh-small
Finished reading data. 0.0209197 sec. Threads: 1
Time to create L: 0.00235761
nonzeros in L: 16952
Time to create S: 0.0124919
nonzeros in S: 8826
iteration 1 0.00117758 secs. 
iteration 2 0.00624599 secs. 
iteration 3 0.00122534 secs. 
iteration 4 0.00559134 secs. 
iteration 5 0.00130012 secs. 
iteration 6 0.00550294 secs. 
iteration 7 0.00128664 secs. 
iteration 8 0.0053474 secs. 
iteration 9 0.00127654 secs. 
iteration 10 0.00529018 secs. 
iteration 11 0.00129985 secs. 
iteration 12 0.00518176 secs. 
iteration 13 0.00129806 secs. 
iteration 14 0.00522962 secs. 
iteration 15 0.00127843 secs. 
iteration 16 0.00516843 secs. 
iteration 17 0.00129001 secs. 
iteration 18 0.00514938 secs. 
iteration 19 0.00126501 secs. 
iteration 20 0.00506839 secs. 
iteration 21 0.00127944 secs. 
iteration 22 0.00503653 secs. 
itera

In [ ]:
for i in range(res.shape[0]):
    for j in range(res.shape[1]):
        res.iloc[i][j]=round(res.iloc[i][j],3)
res
pd.to_pickle(res,"results_t1.pkl")

In [ ]:
res=pd.read_pickle("results_v1.pkl")
res

In [ ]:
res=pd.read_pickle("results_synth.pkl")
res

In [ ]:
res=pd.read_pickle("results_t1.pkl")
res

In [ ]:
res=pd.read_pickle("results_t2.pkl")
res

In [ ]:
res=pd.read_pickle("results_t3.pkl")
res

In [ ]:
res=pd.read_pickle("results_t4.pkl")
res

In [ ]:
res=pd.read_pickle("results_danai.pkl")
res

In [ ]:
res=pd.read_pickle("results_gwave.pkl")
res

In [ ]:
res=pd.read_pickle("results_Ginx.pkl")
res

In [ ]:
res=pd.read_pickle("results_Ginx1.pkl")
res

In [ ]:
res=pd.read_pickle("results_scone.pkl")
res